# [Hate Speech Identification Shared Task](https://multihate.github.io/): Subtask 1A at [BLP Workshop](https://blp-workshop.github.io/) @IJCNLP-AACL 2025

This shared task is designed to identify the type of hate, its severity, and the targeted group from social media content. The goal is to develop robust systems that advance research in this area.

In this subtask, given a Bangla text collected from YouTube comments, categorize whether it contains abusive, sexism, religious hate, political hate, profane, or none.

### Downloading dataset from github

In [2]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_test.tsv

--2025-09-10 16:27:44--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8002036 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1A_train.tsv.1’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.08s   

2025-09-10 16:27:44 (98.1 MB/s) - ‘blp25_hatespeech_subtask_1A_train.tsv.1’ saved [8002036/8002036]

--2025-09-10 16:27:44--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubuserconten

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [3]:
!pip install transformers
!pip install datasets
!pip install evaluate
# !pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


#### importing required libraries and setting up logger

In [4]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Defining the training, validation, and test data

In [5]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_test.tsv'

### Disable wandb

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Setting up the training parameters

In [7]:
# model to load elsewhere: "ai4bharat/IndicBERTv2-MLM-only"
from transformers import TrainingArguments

training_args = TrainingArguments(
    # === keep directory name unchanged ===
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,

    # === core training ===
    learning_rate=2e-5,
    num_train_epochs=4,                 # same as before
    per_device_train_batch_size=16,     # unchanged
    per_device_eval_batch_size=16,      # unchanged
    gradient_accumulation_steps=1,      # unchanged

    # === saving & checkpointing ===
    save_strategy="epoch",              # save checkpoints each epoch
    save_total_limit=2,                 # keep last 2 checkpoints

    # === optimization recipe ===
    lr_scheduler_type="linear",
    warmup_ratio=0.01,                  # unchanged; works fine for IndicBERT v2
    weight_decay=0.01,

    # === misc / perf ===
    remove_unused_columns=False,
    report_to=None,                     # disable TensorBoard/W&B logging
    seed=42,
    fp16=True,                          # if GPU supports (Ampere+)

    # === distributed (if running with DDP) ===
    local_rank=1,
    ddp_find_unused_parameters=False,
)

# keep your variables
max_train_samples = None
max_eval_samples = None
max_predict_samples = None

# sequence & batch
max_seq_length = 512   # unchanged
batch_size = 16        # unchanged

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=False,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=Inter

#### Defining the Model

In [9]:
model_name = 'microsoft/mdeberta-v3-base'

#### setting the random seed

In [10]:
set_seed(training_args.seed)

#### Loading data files

In [11]:
l2id = {'None': 0, 'Religious Hate': 1, 'Sexism': 2, 'Political Hate': 3, 'Profane': 4, 'Abusive': 5}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

INFO:__main__:loading a local file for train
INFO:__main__:loading a local file for validation
INFO:__main__:loading a local file for test


In [12]:
len(test_df['id'])

10200

##### Extracting number of unique labels

In [13]:
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

[0, 5, 4, 1, 3, 2]


### Loading Pretrained Configuration, Tokenizer and Model

In [14]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

[INFO|configuration_utils.py:765] 2025-09-10 16:29:02,706 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--mdeberta-v3-base/snapshots/a0484667b22365f84929a935b5e50a51f71f159d/config.json
[INFO|configuration_utils.py:839] 2025-09-10 16:29:02,717 >> Model config DebertaV2Config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-07,
  "legacy": true,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

[INFO|configuration_utils.py:765] 2025-09-10 16:29:02,834 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--mdeberta-v3-base/snapshots/a0484667b22365f84929a935b5e50a51f71f159d/config.json
[INFO|configuration_utils.py:839] 2025-09-10 16:29:02,836 >> Model config DebertaV2Config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "legacy": true,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true

...

[INFO|tokenization_utils_base.py:2068] 2025-09-10 16:29:04,113 >> loading file spm.model from cache at /root/.cache/huggingface/hub/models--microsoft--mdeberta-v3-base/snapshots/a0484667b22365f84929a935b5e50a51f71f159d/spm.model
[INFO|tokenization_utils_base.py:2068] 2025-09-10 16:29:04,114 >> loading file tokenizer.json from cache at None
[INFO|tokenization_utils_base.py:2068] 2025-09-10 16:29:04,117 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2068] 2025-09-10 16:29:04,118 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2068] 2025-09-10 16:29:04,119 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--mdeberta-v3-base/snapshots/a0484667b22365f84929a935b5e50a51f71f159d/tokenizer_config.json
[INFO|tokenization_utils_base.py:2068] 2025-09-10 16:29:04,121 >> loading file chat_template.jinja from cache at None
[INFO|configuration_utils.py:765] 2025-09-10 16:2

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

[INFO|modeling_utils.py:1280] 2025-09-10 16:29:34,183 >> loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--microsoft--mdeberta-v3-base/snapshots/a0484667b22365f84929a935b5e50a51f71f159d/pytorch_model.bin
[INFO|safetensors_conversion.py:61] 2025-09-10 16:29:34,305 >> Attempting to create safetensors variant
[INFO|safetensors_conversion.py:74] 2025-09-10 16:29:34,644 >> Safetensors PR exists


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

[INFO|modeling_utils.py:5711] 2025-09-10 16:29:38,668 >> Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['deberta.embeddings.word_embeddings._weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a 

#### Preprocessing the raw_datasets

In [15]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/35522 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10200 [00:00<?, ? examples/s]

#### Finalize the training data for training the model

In [16]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [17]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 35522
})

#### Finalize the development/evaluation data for evaluating the model

In [18]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [19]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [20]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

INFO:__main__:Sample 7296 of the training set: {'id': 660, 'text': 'সরকারের দায়িত্বপ্রাপ্ত সংস্থা সকল বাণিজ্যিক ভবন গুলোকে বছরে অন্তত একবার পরিদর্শন করে রিপোর্ট প্রদান করবেন একটা বাণিজ্যিক ভবনে নিরাপত্তার সকল ব্যবস্থা থাকা উচিত কিন্তু এই সব সম্ভবের দেশে কিছুই করা হয় না এই দেশে মানুষের জীবনের কোন দাম নেই তাই কেউ কোনকিছুর তদারকিরও দরকার মনে করে না', 'label': 3, 'input_ids': [1, 260, 16716, 1199, 260, 34323, 69786, 95907, 9040, 42940, 2403, 49671, 4831, 82860, 79265, 5996, 9599, 67135, 260, 200926, 2814, 6224, 6054, 88696, 3718, 2593, 3913, 6947, 44825, 1153, 2142, 68489, 63295, 2997, 39877, 2213, 18499, 2593, 3460, 4831, 82860, 79265, 5996, 9599, 1808, 3425, 4382, 58673, 93041, 1277, 2403, 49671, 260, 48275, 8820, 3533, 5434, 40746, 260, 13679, 14051, 260, 5289, 260, 8910, 17104, 7807, 38138, 260, 34738, 6174, 11693, 1414, 260, 5400, 5164, 3216, 260, 5289, 260, 34738, 260, 17711, 1199, 44715, 1199, 260, 18697, 17695, 2338, 260, 23975, 4738, 1414, 24035, 12697, 260, 18697, 10305, 11693, 1

#### Get the metric function `accuracy`

In [21]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)

    acc  = accuracy_score(p.label_ids, preds)
    f1   = f1_score(p.label_ids, preds, average="micro")   # এখানে micro
    prec = precision_score(p.label_ids, preds, average="micro")
    rec  = recall_score(p.label_ids, preds, average="micro")

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": prec,
        "recall": rec,
    }

#### Data Collator

In [23]:
data_collator = default_data_collator

#### Initialize our Trainer

In [24]:
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-1383425655.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
[INFO|trainer.py:757] 2025-09-10 16:30:08,657 >> Using auto half precision backend


#### Training our model

In [26]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[WARNING|trainer.py:985] 2025-09-10 16:30:08,676 >> The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
[INFO|trainer.py:2523] 2025-09-10 16:30:09,387 >> ***** Running training *****
[INFO|trainer.py:2524] 2025-09-10 16:30:09,387 >>   Num examples = 35,522
[INFO|trainer.py:2525] 2025-09-10 16:30:09,388 >>   Num Epochs = 4
[INFO|trainer.py:2526] 2025-09-10 16:30:09,389 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2529] 2025-09-10 16:30:09,390 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:2530] 2025-09-10 16:30:09,390 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2531] 2025-09-10 16:30:09,391 >>   Total optimization steps = 8,884
[INFO|trainer.py:2532] 2025-09-10 16:30:09,392 >>   Number of trainable parameters

Step,Training Loss
500,1.061800
1000,0.886200
1500,0.827400
2000,0.786200
2500,0.717200
3000,0.680700
3500,0.710300
4000,0.693000
4500,0.677200
5000,0.600300


[INFO|trainer.py:4289] 2025-09-10 16:38:33,686 >> Saving model checkpoint to ./distilBERT_m/checkpoint-2221
[INFO|configuration_utils.py:491] 2025-09-10 16:38:33,690 >> Configuration saved in ./distilBERT_m/checkpoint-2221/config.json
[INFO|modeling_utils.py:4297] 2025-09-10 16:39:41,343 >> Model weights saved in ./distilBERT_m/checkpoint-2221/model.safetensors
[INFO|tokenization_utils_base.py:2563] 2025-09-10 16:39:41,349 >> tokenizer config file saved in ./distilBERT_m/checkpoint-2221/tokenizer_config.json
[INFO|tokenization_utils_base.py:2572] 2025-09-10 16:39:41,357 >> Special tokens file saved in ./distilBERT_m/checkpoint-2221/special_tokens_map.json
[INFO|trainer.py:4289] 2025-09-10 16:49:38,960 >> Saving model checkpoint to ./distilBERT_m/checkpoint-4442
[INFO|configuration_utils.py:491] 2025-09-10 16:49:38,965 >> Configuration saved in ./distilBERT_m/checkpoint-4442/config.json
[INFO|modeling_utils.py:4297] 2025-09-10 16:50:48,081 >> Model weights saved in ./distilBERT_m/checkp

#### Saving the tokenizer too for easy upload

In [27]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:4289] 2025-09-10 17:12:12,676 >> Saving model checkpoint to ./distilBERT_m/
[INFO|configuration_utils.py:491] 2025-09-10 17:12:12,684 >> Configuration saved in ./distilBERT_m/config.json
[INFO|modeling_utils.py:4297] 2025-09-10 17:13:06,890 >> Model weights saved in ./distilBERT_m/model.safetensors
[INFO|tokenization_utils_base.py:2563] 2025-09-10 17:13:06,894 >> tokenizer config file saved in ./distilBERT_m/tokenizer_config.json
[INFO|tokenization_utils_base.py:2572] 2025-09-10 17:13:06,895 >> Special tokens file saved in ./distilBERT_m/special_tokens_map.json


***** train metrics *****
  epoch                    =        4.0
  total_flos               =  8704824GF
  train_loss               =     0.6742
  train_runtime            = 0:41:59.25
  train_samples            =      35522
  train_samples_per_second =     56.401
  train_steps_per_second   =      3.526


#### Evaluating our model on validation/development data

In [28]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
#print(metrics)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

INFO:__main__:*** Evaluate ***
[INFO|trainer.py:4623] 2025-09-10 17:13:07,014 >> 
***** Running Evaluation *****
[INFO|trainer.py:4625] 2025-09-10 17:13:07,015 >>   Num examples = 2512
[INFO|trainer.py:4628] 2025-09-10 17:13:07,016 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.7114
  eval_f1                 =     0.7114
  eval_loss               =     0.7752
  eval_precision          =     0.7114
  eval_recall             =     0.7114
  eval_runtime            = 0:00:08.18
  eval_samples            =       2512
  eval_samples_per_second =    307.063
  eval_steps_per_second   =     19.191


#### epchos - 03 --> 72.73
#### epchos - 04 --> 72.37

### Predecting the test data

In [29]:
id2l = {v: k for k, v in l2id.items()}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = "subtask_1A.tsv"
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\tmodel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\t{model_name}\n")

INFO:__main__:*** Predict ***
[INFO|trainer.py:4623] 2025-09-10 17:13:15,212 >> 
***** Running Prediction *****
[INFO|trainer.py:4625] 2025-09-10 17:13:15,213 >>   Num examples = 10200
[INFO|trainer.py:4628] 2025-09-10 17:13:15,215 >>   Batch size = 16


INFO:__main__:***** Predict results *****


In [30]:
ids[0]

12764

#### Saving the model into card

In [31]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:456] 2025-09-10 17:13:49,690 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7113853503184714}, {'name': 'F1', 'type': 'f1', 'value': 0.7113853503184714}, {'name': 'Precision', 'type': 'precision', 'value': 0.7113853503184714}, {'name': 'Recall', 'type': 'recall', 'value': 0.7113853503184714}]}


In [32]:
!zip subtask_1A.zip subtask_1A.tsv

  adding: subtask_1A.tsv (deflated 89%)
